In [1]:
import os
import datetime
import pandas as pd
import numpy as np

In [2]:
lizon_id = 69715

In [2]:
Data='/home/kate/Projects/eva/Data'

Messages_filename='Main/Messages.csv'
Messages_full_filename=os.path.join(Data, Messages_filename)

OriginalMessages_filename='Main/OriginalMessages.csv'
OriginalMessages_full_filename=os.path.join(Data, OriginalMessages_filename)

Authors_filename='Main/Authors.csv'
Authors_full_filename=os.path.join(Data, Authors_filename)

ta_extension_filename='Main/TextAttributes_ext.csv'
ta_extension_full_filename=os.path.join(Data, ta_extension_filename)


ft_filename='lizon_data_for_finetuning.csv'
ft_full_filename=os.path.join(Data, ft_filename)

ft_filename_test='Subprojects/lizon/lizon_clone_data_for_test.csv'
ft_full_filename_test=os.path.join(Data, ft_filename_test)

SimilarAuthors_filename='Subprojects/lizon/lizon_similar_authors.csv'
SimilarAuthors_full_filename=os.path.join(Data, SimilarAuthors_filename)

all_lizon_messages_filename='Subprojects/lizon/all_lizon_and_clones_messages.csv'
all_lizon_messages_full_filename=os.path.join(Data, all_lizon_messages_filename)

In [ ]:
Messages = pd.read_csv(Messages_full_filename, error_bad_lines=False, index_col=False)

In [ ]:
AMessages = Messages[Messages['Author_Id']!=0].copy(deep=True)

In [ ]:
del Messages

In [ ]:
len(AMessages)

## Messages, no less then 25 words, from all topics where lizon present

In [ ]:
topic_id_with_lizon=Messages[Messages['Author_Id']==lizon_id]['Topic_Id'].unique()

In [ ]:
len(topic_id_with_lizon)

In [ ]:
AMessages=Messages[Messages['Topic_Id'].isin(topic_id_with_lizon)].copy(deep=True)

In [ ]:
AMessages=AMessages[AMessages['Author_Id']!=0]

In [ ]:
AMessages= AMessages[AMessages['message_words']>=25]

In [ ]:
len(AMessages)

In [ ]:
AMessages['target']=['lizon' if x == lizon_id else 'Other' for x in AMessages['Author_Id']]

In [ ]:
AMessages.groupby(['target']).size().reset_index(name='counts').sort_values('counts', ascending=False)

In [ ]:
#AMessages=AMessages[['Message_Id','Author_Id','author', 'message','original_message','message_words','target']]

In [ ]:
AMessages.to_csv(ft_full_filename, header=True, index=False)

## Dataset to find lizon's clones (model application)

### Authors to check

In [4]:
Authors = pd.read_csv(Authors_full_filename, error_bad_lines=False, index_col=False)

In [5]:
Authors.head()

,Author_Id,author,cnt_messages,First_Message_Id,Last_Message_Id,first_message_ts,last_message_ts,cnt_topics_started,avg_immediate_childs,avg_childs,...,avg_messages_with_questions,avg_ASW,avg_PLW,avg_TTR,avg_TTR_A,avg_TTR_N,avg_TTR_V,avg_NAV,avg_UNAV,avg_message_words_with_nl
0,23,volv ♧ Clair,1535,67913773,101696362,2011-07-06 11:00:00,2020-11-09 23:07:00,140,1.013681,3.510098,...,0.159609,2.110231,0.131055,0.695840,0.477450,0.787899,0.696991,0.963938,1.471982,11.055375
1,27,Oxygen 27 F*,16,97031390,102288727,2018-05-17 14:17:00,2021-02-19 17:01:00,0,0.312500,0.812500,...,0.000000,1.931435,0.088466,0.821663,0.675000,0.907087,0.750000,1.144587,1.690625,23.562500
2,32,Редакция Евы,784,80470511,96142872,2013-04-29 13:46:00,2017-12-11 20:44:00,758,2.332908,5.241071,...,0.237245,2.126801,0.158996,0.899764,0.894424,0.881867,0.960929,1.854450,3.460838,39.991071
3,35,Малефисента ⚜**,3415,51064403,101933631,2009-11-02 14:26:00,2020-12-21 17:49:00,4,0.720059,2.950220,...,0.267643,1.868171,0.094348,0.815487,0.540761,0.887042,0.836398,1.331377,1.755030,19.369253
4,36,Доктор Филатов SD*,4992,81110399,101760074,2013-06-18 17:11:00,2020-11-20 18:00:00,8,0.860377,3.353566,...,0.223758,2.066710,0.143977,0.864907,0.783424,0.867137,0.894132,1.730154,2.140106,55.311098


In [6]:
Authors.columns

Index(['Author_Id', 'author', 'cnt_messages', 'First_Message_Id',
       'Last_Message_Id', 'first_message_ts', 'last_message_ts',
       'cnt_topics_started', 'avg_immediate_childs', 'avg_childs', 'TotalDays',
       'DaysWithMessages', 'avg_messages_per_day', 'Period1', 'Period2',
       'Period3', 'avg_messages_per_day_Period1',
       'avg_messages_per_day_Period2', 'avg_messages_per_day_Period3',
       'PeriodDailyPeriodicity', 'dawn', 'early morning', 'late morning',
       'noon', 'afternoon', 'evening', 'night', 'midnight',
       'DayPartPeriodicity', 'Period1_cnt_messages', 'Period2_cnt_messages',
       'Period3_cnt_messages', '1 - dawn', '1 - early morning',
       '1 - late morning', '1 - noon', '1 - afternoon', '1 - evening',
       '1 - night', '1 - midnight', '2 - dawn', '2 - early morning',
       '2 - late morning', '2 - noon', '2 - afternoon', '2 - evening',
       '2 - night', '2 - midnight', '3 - dawn', '3 - early morning',
       '3 - late morning', '3 - noon', '

In [7]:
Authors['first_message_ts']=pd.to_datetime(Authors['first_message_ts'], format='%Y-%m-%d %H:%M:%S')
Authors['last_message_ts']=pd.to_datetime(Authors['last_message_ts'], format='%Y-%m-%d %H:%M:%S')

In [8]:
#lizon's last message
Authors[Authors['Author_Id']==lizon_id]['last_message_ts']

3485   2010-12-07 23:18:00
Name: last_message_ts, dtype: datetime64[ns]

In [9]:
#All new authors after the date
len(Authors[Authors['first_message_ts']>'2010-12-07 23:18:00'])

10735

In [10]:
#Authors posted after the last lizon's date with reasonable number of messages
#5000 because I would remember an author with more messages
#100 because with less messages authors write as Anonymous
#TotalDays>1 because authors with 100+ messages in very few days are spamers most likely and they are banned
len(Authors[((Authors['first_message_ts']>'2010-12-07 23:18:00') & (Authors['TotalDays']>1) & (Authors['cnt_messages']>100) & (Authors['cnt_messages']<5000))])

670

In [11]:
Authors = Authors[((Authors['Author_Id']==lizon_id) | ((Authors['first_message_ts']>'2010-12-07 23:18:00') & (Authors['TotalDays']>1) & (Authors['cnt_messages']>100) & (Authors['cnt_messages']<5000)))]

### Can we limit the number of authors to check even more?

In [12]:
from sklearn.metrics.pairwise import cosine_similarity

In [13]:
columns_to_compare=[
'fraction_of_tneg_messages',
'fraction_of_tpos_messages',
'fraction_of_Large_Topics',
'fraction_of_messages_with_immediate_responses',
'avg_immediate_responses',
'fraction_of_messages_with_discussion',
'avg_responses',
'fraction_of_neg_rersponses',
'fraction_of_pos_rersponses',
'fraction_of_messages_with_emojis',
'fraction_of_messages_with_images',
'fraction_of_messages_with_links',
'fraction_of_messages_with_excessive_exclamations',
'fraction_of_messages_with_excessive_questions',
'fraction_of_messages_with_excessive_other',
'fraction_of_medium_with_commas',
'avg_num_commas',
'avg_sent_len',
'avg_num_sent',
'avg_ASW',
'avg_PLW',
'avg_fraction_of_Adj',
'avg_fraction_of_Verbs',
'avg_fraction_of_Nouns',
'avg_TTR',
'avg_TTR_A',
'avg_TTR_N',
'avg_TTR_V',
'avg_NAV',
'avg_UNAV']

In [15]:
lizon=np.array(Authors[Authors['Author_Id']==lizon_id][columns_to_compare])
lizon

array([[1.06122449e+00, 4.49372057e+00, 3.56223176e-01, 1.11000000e+02,
        1.12244898e-01, 5.65149137e-02, 4.37205651e-01, 2.34693878e-01,
        1.92307692e-01, 9.49764521e-02, 9.89010989e-02, 3.76766091e-02,
        4.60753532e-01, 1.67189953e-01, 1.66405024e-01, 7.53532182e-02,
        0.00000000e+00, 0.00000000e+00, 1.42072214e-01, 3.31483516e+01,
        1.27629513e+00, 1.21821036e+00, 5.77456247e+00, 1.72684458e-02,
        3.92464678e-02, 2.35478807e-03, 6.82888540e-02, 2.98273155e-01,
        1.82822218e+00, 9.73044663e-02, 8.22116815e-01, 6.67332157e-01,
        8.60415825e-01, 8.31225405e-01, 1.53742940e+00, 1.77914751e+00,
        3.41718995e+01]])

In [16]:
others=np.array(Authors[columns_to_compare])
others

array([[ 1.01368078,  3.51009772,  0.0887522 , ...,  0.96393826,
         1.47198227, 11.05537459],
       [ 2.33290816,  5.24107143,  0.31298162, ...,  1.8544502 ,
         3.46083783, 39.99107143],
       [ 0.8603766 ,  3.35356571,  0.24594395, ...,  1.73015392,
         2.1401055 , 55.31109776],
       ...,
       [ 0.6352459 ,  3.66803279,  0.91304348, ...,  1.30198559,
         1.76042884, 18.23770492],
       [ 0.95049505,  9.08910891,  0.54545455, ...,  1.78743686,
         1.840156  , 47.26732673],
       [ 0.8597561 ,  3.45731707,  1.33333333, ...,  1.22108971,
         1.63047185, 17.5       ]])

In [17]:
result=cosine_similarity(lizon,others)

In [18]:
Authors['liz_sim']=result[0]

In [20]:
SimilarAuthors = Authors[Authors['liz_sim']>0.96].sort_values(['liz_sim'],ascending=False)

In [21]:
SimilarAuthors['Author_Id'].nunique()

465

In [22]:
SimilarAuthors.to_csv(SimilarAuthors_full_filename, header=True, index=False)

In [23]:
SimilarAuthors[SimilarAuthors['Author_Id'].isin([69715, 711697,475085,472696,731728])]

,Author_Id,author,cnt_messages,First_Message_Id,Last_Message_Id,first_message_ts,last_message_ts,cnt_topics_started,avg_immediate_childs,avg_childs,...,avg_ASW,avg_PLW,avg_TTR,avg_TTR_A,avg_TTR_N,avg_TTR_V,avg_NAV,avg_UNAV,avg_message_words_with_nl,liz_sim
3485,69715,lizon **,1274,49189796,62257633,2009-08-28 11:50:00,2010-12-07 23:18:00,19,1.061224,4.493721,...,1.828222,0.097304,0.822117,0.667332,0.860416,0.831225,1.537429,1.779148,34.171900,1.000000
19739,731728,newyorck D,2142,100170690,103317149,2020-02-17 15:43:00,2021-09-16 20:49:00,32,1.035481,3.554622,...,1.784620,0.091653,0.793863,0.683340,0.828765,0.795792,1.546820,2.063420,43.940243,0.995389
19218,711697,Lizonn +,136,98135519,98598759,2018-12-29 16:55:00,2019-03-28 23:41:00,2,1.139706,3.632353,...,1.738783,0.085598,0.770215,0.688240,0.780245,0.806089,1.591114,1.983785,51.845588,0.985736
17566,472696,Кассиапея D,1093,89966258,98530888,2015-06-24 22:40:00,2019-03-16 15:05:00,15,1.116194,4.318390,...,1.788286,0.097028,0.803670,0.709322,0.828456,0.810720,1.592780,1.945643,53.623056,0.983421
17627,475085,Элиззи H*,3317,90184059,98561081,2015-07-26 22:11:00,2019-03-21 21:20:00,44,1.109135,4.608683,...,1.775261,0.093371,0.805636,0.713185,0.828437,0.818118,1.655201,1.978193,65.477841,0.962142


In [24]:
SimilarAuthors.tail()

,Author_Id,author,cnt_messages,First_Message_Id,Last_Message_Id,first_message_ts,last_message_ts,cnt_topics_started,avg_immediate_childs,avg_childs,...,avg_ASW,avg_PLW,avg_TTR,avg_TTR_A,avg_TTR_N,avg_TTR_V,avg_NAV,avg_UNAV,avg_message_words_with_nl,liz_sim
7668,135565,Иона **K**,584,64358144,78026132,2011-02-19 03:48:00,2012-11-26 15:37:00,17,0.681507,1.820205,...,1.840014,0.091946,0.864496,0.819727,0.852036,0.896759,1.817031,1.872381,63.815068,0.960923
18699,535878,Dilasa H*,283,95954391,103126625,2017-11-12 10:50:00,2021-08-04 13:57:00,0,0.597173,1.826855,...,1.899906,0.086897,0.620923,0.343051,0.740642,0.622264,0.758347,1.073574,10.318021,0.960658
6763,121007,Cветла +,1516,97921078,102571947,2018-11-18 10:46:00,2021-04-15 01:15:00,0,0.649736,4.185356,...,2.030650,0.112618,0.711570,0.431047,0.777421,0.777554,0.980201,1.259213,9.883245,0.960594
5742,105871,Арисоль KF**,307,73810035,98412060,2012-03-18 00:24:00,2019-02-20 22:58:00,3,0.960912,3.244300,...,1.952808,0.107603,0.878427,0.797343,0.890229,0.891605,1.739197,2.142288,64.055375,0.960415
15799,351481,razdor KF**,2358,73364975,98149372,2012-02-27 17:11:00,2019-01-02 19:05:00,4,0.876590,3.012299,...,2.024916,0.124350,0.869615,0.803953,0.870158,0.896494,1.782647,1.994565,64.147583,0.960312


In [25]:
SimilarAuthors.head()

,Author_Id,author,cnt_messages,First_Message_Id,Last_Message_Id,first_message_ts,last_message_ts,cnt_topics_started,avg_immediate_childs,avg_childs,...,avg_ASW,avg_PLW,avg_TTR,avg_TTR_A,avg_TTR_N,avg_TTR_V,avg_NAV,avg_UNAV,avg_message_words_with_nl,liz_sim
3485,69715,lizon **,1274,49189796,62257633,2009-08-28 11:50:00,2010-12-07 23:18:00,19,1.061224,4.493721,...,1.828222,0.097304,0.822117,0.667332,0.860416,0.831225,1.537429,1.779148,34.171900,1.000000
4392,84248,Вайс ♤ Clair,210,83225066,102672780,2013-11-30 01:52:00,2021-05-06 06:51:00,2,1.019048,4.485714,...,1.934925,0.100307,0.886657,0.761528,0.917742,0.890403,1.684578,2.116749,33.923810,0.999970
16190,372195,Мама Коши KF*,114,90691789,103004527,2015-10-01 08:14:00,2021-07-10 00:04:00,6,1.122807,3.885965,...,1.960179,0.097535,0.845605,0.716656,0.877476,0.842552,1.551251,1.893374,33.350877,0.999920
16737,407082,Salvatore ♕ ♕ ♕ SD*,106,81870905,103098360,2013-08-28 09:40:00,2021-07-29 17:04:00,9,1.226415,3.594340,...,1.747303,0.092109,0.798720,0.644356,0.845180,0.795456,1.524547,1.911942,33.066038,0.999887
16440,388272,Старая Ведьма D**,178,93562793,103099333,2016-10-15 21:18:00,2021-07-29 20:35:00,1,0.702247,4.089888,...,1.838805,0.088387,0.802443,0.607183,0.846396,0.840087,1.489347,1.523778,32.612360,0.999868


## Messages to analyze

In [4]:
SimilarAuthors = pd.read_csv(SimilarAuthors_full_filename, error_bad_lines=False, index_col=False)

In [5]:
ta_ext = pd.read_csv(ta_extension_full_filename, error_bad_lines=False, index_col=False) 

In [6]:
OriginalMessages = pd.read_csv(OriginalMessages_full_filename, error_bad_lines=False, index_col=False)

In [7]:
OriginalMessages.columns

Index(['Message_Id', 'Timestamp', 'Topic_1st_Message', 'Parent_Id',
       'Author_Id', 'author', 'Topic_Id', 'Topic', 'Chapter_Id', 'Chapter',
       'original_message', 'cnt_childs', 'cnt_immediate_childs'],
      dtype='object')

In [8]:
ta_ext.columns

Index(['Message_Id', 'Timestamp', 'Topic_1st_Message', 'Parent_Id',
       'Author_Id', 'author', 'Topic_Id', 'Topic', 'Chapter_Id', 'Chapter',
       'message_characters', 'message_words', 'emojis', 'images', 'links',
       'original_paragraphs', 'actual_paragraphs', 'avg_paragraph_characters',
       'avg_paragraph_words', 'cnt_childs', 'cnt_immediate_childs',
       'flg_excessive_exclamations', 'flg_excessive_questions',
       'flg_excessive_other', 'num_Adj', 'num_unique_Adj', 'num_Nouns',
       'num_unique_Nouns', 'num_Verb', 'num_unique_Verb', 'num_tokens',
       'num_unique_tokens', 'num_syllables', 'num_long_words',
       'num_unique_long_words', 'num_commas', 'num_exclamations',
       'num_questions', 'num_words', 'ASW', 'PLW', 'TTR', 'TTR_A', 'TTR_N',
       'TTR_V', 'NAV', 'UNAV', 'fraction_of_commas',
       'fraction_of_exclamations', 'fraction_of_questions', 'fraction_of_Adj',
       'fraction_of_Nouns', 'fraction_of_Verbs', 'diff'],
      dtype='object')

In [9]:
#How many 25+ words messages belong to the similar authors?
SimilarMessagesIds = ta_ext[((ta_ext['Author_Id'].isin(SimilarAuthors['Author_Id'].tolist())) & (ta_ext['Author_Id']!=lizon_id) & (ta_ext['message_words']>=25))]['Message_Id']

In [10]:
len(OriginalMessages[OriginalMessages['Message_Id'].isin(SimilarMessagesIds.tolist())])

123724

In [13]:
Columns_to_test_dataset=['Message_Id', 'Timestamp', 'Parent_Id','Author_Id',  'Topic_Id', 'Topic', 'original_message']

In [14]:
OriginalMessages[OriginalMessages['Message_Id'].isin(SimilarMessagesIds.tolist())][Columns_to_test_dataset].to_csv(ft_full_filename_test, header=True, index=False)

In [6]:
Authors[Authors['author']=='Lizon " **K**'].head()

,Author_Id,author,cnt_messages,First_Message_Id,Last_Message_Id,first_message_ts,last_message_ts,cnt_topics_started,avg_immediate_childs,avg_childs,...,avg_messages_with_questions,avg_ASW,avg_PLW,avg_TTR,avg_TTR_A,avg_TTR_N,avg_TTR_V,avg_NAV,avg_UNAV,avg_message_words_with_nl
14771,300514,"Lizon "" **K**",955,62997968,78899997,2011-01-04 14:22:00,2013-01-19 20:38:00,30,1.062827,4.025131,...,0.216754,1.809122,0.093493,0.823809,0.708906,0.846763,0.836523,1.618202,1.85228,42.070157


## All lizon's posts from clones

In [3]:
lizon_ids=[69715,
300514,
100899,
424711,
472696,
475085,
711697,
731728]

In [4]:
OriginalMessages = pd.read_csv(OriginalMessages_full_filename, error_bad_lines=False, index_col=False)

In [5]:
lizon_original_messages=OriginalMessages[OriginalMessages['Author_Id'].isin(lizon_ids)].copy(deep=True)

In [6]:
len(lizon_original_messages)

9778

In [7]:
del OriginalMessages

In [8]:
Messages = pd.read_csv(Messages_full_filename, error_bad_lines=False, index_col=False)

In [9]:
lizon_messages=Messages[Messages['Author_Id'].isin(lizon_ids)][['Message_Id','message']].copy(deep=True)

In [11]:
len(lizon_messages)

9778

In [10]:
del Messages

In [12]:
lizon_messages=pd.merge(lizon_original_messages, lizon_messages, left_on='Message_Id', right_on='Message_Id', how='inner')

In [15]:
lizon_messages=lizon_messages[['Message_Id', 'Timestamp', 'Topic_1st_Message', 'Parent_Id',
       'Author_Id', 'author', 'Topic_Id', 'Topic', 'Chapter_Id', 'Chapter',
       'original_message', 'message', 'cnt_childs', 'cnt_immediate_childs']]

In [16]:
del lizon_original_messages

### To unify author names (can vary in posts)

In [17]:
Authors = pd.read_csv(Authors_full_filename, error_bad_lines=False, index_col=False)

In [18]:
lizon_messages=pd.merge(lizon_messages, Authors[['Author_Id','author']],left_on='Author_Id', right_on='Author_Id', how='inner')

In [20]:
lizon_messages=lizon_messages[['Message_Id', 'Timestamp', 'Topic_1st_Message', 'Parent_Id',
       'Author_Id', 'author_y', 'Topic_Id', 'Topic', 'Chapter_Id', 'Chapter',
       'original_message', 'message', 'cnt_childs', 'cnt_immediate_childs']]
lizon_messages.columns=['Message_Id', 'Timestamp', 'Topic_1st_Message', 'Parent_Id',
       'Author_Id', 'author', 'Topic_Id', 'Topic', 'Chapter_Id', 'Chapter',
       'original_message', 'message', 'cnt_childs', 'cnt_immediate_childs']

In [21]:
len(lizon_messages)

9778

In [22]:
lizon_messages.head()

,Message_Id,Timestamp,Topic_1st_Message,Parent_Id,Author_Id,author,Topic_Id,Topic,Chapter_Id,Chapter,original_message,message,cnt_childs,cnt_immediate_childs
0,56799126,2010-05-12 22:03:00,N,0,69715,lizon **,2281736,как часто вы покупаете ребенку игрушки?,137,Детская психология и развитие,"<div class=""body"">Каждую неделю, на барахолках...","Каждую неделю, на барахолках, где-то раз в мес...",1,1
1,57921402,2010-07-01 12:38:00,N,0,69715,lizon **,2322960,про негра,137,Детская психология и развитие,"<div class=""body"">Хе, а у нас, где мы жили ран...","Хе, а у нас, где мы жили раньше(свят свят, что...",0,0
2,58468210,2010-08-02 14:42:00,N,0,69715,lizon **,2350112,Как вы ходите с детьми в рестораны?,137,Детская психология и развитие,"<div class=""body"">Начали ходить с дитем в рест...",Начали ходить с дитем в рестораны лет с четыр...,0,0
3,58468241,2010-08-02 14:43:00,N,0,69715,lizon **,2351751,как спят ваши дети в жару,137,Детская психология и развитие,"<div class=""body"">Укладываю в жару спать в отк...",Укладываю в жару спать в открытой палатке в са...,0,0
4,58795095,2010-08-19 22:47:00,Y,0,69715,lizon **,2368698,читаь молитву на ночь в 5 лет-не слишком ли?,137,Детская психология и развитие,"<div class=""body"">У меня муж очень верующий ка...",У меня муж очень верующий католик. Я атеистка....,50,17


In [24]:
all_lizon_messages_full_filename

'/home/kate/Projects/eva/Data/Subprojects/lizon/all_lizon_and_clones_messages.csv'

In [25]:
lizon_messages.to_csv(all_lizon_messages_full_filename, header=True, index=False)

In [3]:
lizon_messages = pd.read_csv(all_lizon_messages_full_filename, error_bad_lines=False, index_col=False)

In [4]:
ta_ext = pd.read_csv(ta_extension_full_filename, error_bad_lines=False, index_col=False)

In [5]:
lizon_messages=pd.merge(lizon_messages, ta_ext[['Message_Id','message_words']],left_on='Message_Id', right_on='Message_Id', how='inner')

In [6]:
len(lizon_messages)

9778

In [7]:
lizon_messages.to_csv(all_lizon_messages_full_filename, header=True, index=False)